In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn  as sns
import geopandas as gpd
import fiona
import shapely
import matplotlib as mpl
from statannotations.Annotator import Annotator
from statannotations.stats.StatTest import StatTest
import pickle
from shapely.geometry import Point, MultiPoint
import scipy
from shapely import wkt
import statsmodels.api as sm


In [ ]:
from shapely.geometry import Polygon

In [ ]:
from pygam import LogisticGAM, s, te

In [ ]:
from functools import reduce
from operator import add


In [ ]:
import libpysal

In [ ]:
import esda

In [ ]:
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
plt.rcParams.update({'font.size': 16})

#### Load datasets

In [ ]:
gdf=gpd.read_file("../Data/Socio_economic_data.geojson")

In [ ]:
# import France outline
fr_outline=gpd.read_file("../Maps/france_outline.geojson")

In [ ]:
# import Allodiality data
allodiality=gpd.read_file("../Data/Allodiality_data.geojson")

In [ ]:
# import prices
prices=gpd.read_file("../Data/Price_data.geojson")

#### Plots and statistical analysis

##### Association with population: restricted to towns (more than 2000 inhabitants)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax=sns.boxplot(data=gdf[gdf.Town], x="Fear", y="logP", order=["No","Yes"], palette=["tab:blue","red"])
pairs=[("No","Yes")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town], x="Fear", y="logP", order=["No","Yes"])
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel(r"$\log_{10}$ Population in towns (1789)")
plt.show()

##### Plot map

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
fr_outline.plot(facecolor="white", edgecolor='black', lw=0.3, ax=ax)
gdf[gdf.Town].plot(column="Fear", marker=".", ax=ax, cmap="coolwarm", s=gdf[gdf.Town]['poptot']/200)
plt.xlim(0.1e6,1.12e6)

##### Correlation with the participation in the 1789 referendum (towns with more than 2000 inhabitants)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax=sns.boxplot(data=gdf[gdf.Town], x="Fear", y="ppar", order=["No","Yes"], palette=["tab:blue","red"])
pairs=[("Yes","No")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town], x="Fear", y="ppar", order=["No","Yes"])
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel(r"Participation in 1793 referendum (towns)")
plt.show()

##### Association with literacy (towns with more than 2000 inhabitants)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax=sns.boxplot(data=gdf[gdf.Town], x="Fear", y="% signed", order=["No","Yes"], palette=["tab:blue","red"])
pairs=[("Yes","No")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town], x="Fear", y="% signed")
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel("Literacy rate (1786 towns)")
plt.show()

In [ ]:
# literacy rate (averaged)
(gdf.poptot*gdf["% signed"]).sum()/gdf.poptot.sum()

In [ ]:
# literacy rate (averaged)
(gdf[gdf.Town].poptot*gdf[gdf.Town]["% signed"]).sum()/gdf[gdf.Town].poptot.sum()

##### Association with income (towns of more than 2000 inhabitants)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax=sns.boxplot(data=gdf[gdf.Town], x="Fear", y="revratio1790", order=["No","Yes"], palette=["tab:blue","red"])
pairs=[("Yes","No")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town], x="Fear", y="revratio1790")
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel("Relative income per person (1790 towns)")
plt.tight_layout()
plt.show()

##### Association with ownership (towns of more than 2000 inhabitants)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax=sns.boxplot(data=gdf[gdf.Town], x="Fear", y="perpropri1790", order=["No","Yes"], palette=["tab:blue","red"])
pairs=[("Yes","No")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town], x="Fear", y="perpropri1790")
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel("% ownership (1790 towns)")
plt.show()

##### Association with wheat prices (towns)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 4))
ax=sns.boxplot(data=gdf[gdf.Town==True], x="Fear", y="price_1789", order=["No","Yes"], palette=["tab:blue","tab:red"])
pairs=[("Yes","No")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town==True], x="Fear", y="price_1789")
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel("Price of wheat in 1789")
plt.tight_layout()
plt.show()

##### Association with price differences (towns)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax=sns.boxplot(data=gdf[gdf.Town==True], x="Fear", y="price_increment", order=["No","Yes"], palette=["tab:blue","tab:red"])
pairs=[("Yes","No")]
annotator = Annotator(ax, pairs, data=gdf[gdf.Town==True], x="Fear", y="price_increment")
# Required descriptors for annotate
custom_long_name = 'T Test'
custom_short_name = 'T Test'
custom_func = scipy.stats.ttest_ind
custom_test = StatTest(custom_func, custom_long_name, custom_short_name)
annotator.configure(test=custom_test, text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.ylabel("Price increment of wheat 1787-1789")
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
prices.plot(column="price_1789", legend=True, ax=ax, legend_kwds={"label": "Price of wheat in 1789"})
plt.xlim(0.1e6,1.12e6)

#### Allodiality

In [ ]:
contingency = pd.crosstab(gdf[gdf.Town]['Alodiality'], gdf[gdf.Town]['Fear'])

In [ ]:
contingency

In [ ]:
fractions=100*contingency["Yes"]/(contingency["Yes"]+contingency["No"])

In [ ]:
df_fr=pd.DataFrame(fractions.reset_index())

In [ ]:
df_fr.columns=["Allodiality","% of towns with fear"]

In [ ]:
my_colors=["yellow","red","tab:green"]

In [ ]:
fig = plt.figure(figsize=(3,4))
ax=sns.barplot(data=df_fr,x="Allodiality", y="% of towns with fear", palette=my_colors, edgecolor="black", order=["NSST","NAST","NTSS"])
# Coordinates for annotations
bar_centers = [p.get_x() + p.get_width() / 2 for p in ax.patches]
bar_heights = [p.get_height() for p in ax.patches]

# Offsets for the significance bars
bar_offset = 1.0
line_height = 0.5

def annotate_sig(ax, x1, x2, y, h, text):
    """Draws a significance bar between two bars with an asterisk."""
    ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c='black')
    ax.text((x1 + x2) / 2, y + h + 0.2, text, ha='center', va='bottom', fontsize=14)
# A vs B
annotate_sig(ax, bar_centers[0], bar_centers[1], max(bar_heights) + bar_offset, line_height, '****')
# B vs C
annotate_sig(ax, bar_centers[1], bar_centers[2], max(bar_heights) + bar_offset + 1., line_height, '****')
# A vs C
annotate_sig(ax, bar_centers[0], bar_centers[2], max(bar_heights) + bar_offset + 3.5, line_height, '****')
plt.ylim(0, max(bar_heights) + bar_offset + 6)  # make room for annotations

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
allodiality.plot(column="Alodiality", ax=ax, cmap="prism", legend=True)
#plt.savefig("alod.pdf")

#### Multiple logistic regression

In [ ]:
df1=gdf[gdf.Town][["logP","revratio1790","perpropri1790","% signed",'ppar',"Fear",'price_1789', 'price_increment','Alodiality']].dropna()

In [ ]:
df1

In [ ]:
def NSST(x):
    if x=="NSST":
        allod=1
    else:
        allod=0
    return allod

In [ ]:
def binarize_fear(x):
    if x=="Yes":
        fear=1
    else:
        fear=0
    return fear

In [ ]:
df1.Fear=df1.Fear.apply(lambda x: binarize_fear(x))

In [ ]:
df1["NSST"]=df1.Alodiality.apply(lambda x: NSST(x))

In [ ]:
X=df1[['logP', 'revratio1790', 'perpropri1790', '% signed', 'ppar',"price_1789","price_increment", "NSST"]]
Y=df1["Fear"]

In [ ]:
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(Y, X)
result = model.fit(method='bfgs', maxiter=500)


# Print the summary
print(result.summary())

In [ ]:
# Get odds ratios
odds_ratios = np.exp(result.params)
# Get 95% CI
conf = result.conf_int()
conf.columns = ['2.5%', '97.5%']
conf_exp = np.exp(conf)
# Combine into a single table
summary_table = pd.concat([odds_ratios, conf_exp], axis=1)
summary_table.columns = ['Odds Ratio', '2.5%', '97.5%']
print(summary_table)


In [ ]:
print(result.pvalues)

#### Control for geography

In [ ]:
y_pred = result.predict(X)

In [ ]:
pearson_residuals = (Y - y_pred) / np.sqrt(y_pred * (1 - y_pred))

In [ ]:
gdf2=gdf[gdf.Town==True][['geometry',"logP","revratio1790","perpropri1790","% signed",'ppar',"Fear", "price_1789","price_increment", "Alodiality"]]

In [ ]:
gdf2.Fear=gdf2.Fear.apply(lambda x: binarize_fear(x))

In [ ]:
gdf2["NSST"]=gdf2.Alodiality.apply(lambda x: NSST(x))

In [ ]:
gdf2=gdf2.dropna()

In [ ]:
gdf2["res"]=pearson_residuals

In [ ]:
gdf2=gdf2[["geometry","res"]]

In [ ]:
coords = list(zip(gdf2.geometry.x, gdf2.geometry.y))
# Define maximum distance for neighbors (you must pick a reasonable threshold)
threshold = 10000  # in the units of your CRS; adjust accordingly!
w = libpysal.weights.DistanceBand(coords, threshold=threshold, binary=True, silence_warnings=True)


In [ ]:
# Calculate Moran's I
moran = esda.Moran(gdf2['res'], w)
# Output the results
print(f"Moran's I: {moran.I}")
print(f"P-value: {moran.p_sim}")

#### Adjust for geographic variations

In [ ]:
gdf1=gdf[gdf.Town==True][['geometry',"Alodiality","logP","revratio1790","perpropri1790","% signed",'ppar',"Fear", "price_1789","price_increment"]]

In [ ]:
gdf1.Fear=gdf1.Fear.apply(lambda x: binarize_fear(x))

In [ ]:
gdf1["NSST"]=gdf1.Alodiality.apply(lambda x: NSST(x))

In [ ]:
gdf1["lat"]=gdf1["geometry"].y
gdf1["lon"]=gdf1["geometry"].x

In [ ]:
gdf1=gdf1.dropna()

In [ ]:
X=gdf1[['logP', 'revratio1790', 'perpropri1790', '% signed', 'ppar',"price_1789","price_increment", "NSST", "lat","lon"]]
Y=gdf1["Fear"].astype(float)

In [ ]:
# Make sure y is 1D array
y_array = Y.values.ravel()



In [ ]:
# Get number of columns (predictors + lon + lat)
n_features = X.shape[1]


In [ ]:
# Indices of longitude and latitude
lon_idx = X.columns.get_loc('lon')
lat_idx = X.columns.get_loc('lat')

In [ ]:
# 1. Create a list of terms first
terms = []

# 2. Add smooth terms for normal predictors
for i in range(X.shape[1]):
    if i not in [lon_idx, lat_idx]:
        terms.append(s(i))  # smooth each non-spatial feature

# 3. Add 2D spatial smoother
terms.append(te(lon_idx, lat_idx))


In [ ]:
# 4. Combine terms correctly (sum them up with +)


combined_terms = reduce(add, terms)

# 5. Fit the model
gam = LogisticGAM(combined_terms, lam=1).fit(X.values, y_array)



In [ ]:
print(gam.summary())



In [ ]:
# Get term indices for non-intercept terms
term_indices = [i for i, t in enumerate(gam.terms) if not t.isintercept]

# Create subplots for each non-intercept term
fig, axs = plt.subplots(len(term_indices), 1, figsize=(8, len(term_indices) * 3))

if len(term_indices) == 1:
    axs = [axs]  # Make sure it's iterable

# Plot partial dependence and confidence intervals
for ax, i in zip(axs, term_indices):
    XX = gam.generate_X_grid(term=i)
    pdep = gam.partial_dependence(term=i, X=XX)
    conf = gam.partial_dependence(term=i, X=XX, width=0.95)

    ax.plot(XX[:, i], pdep, label='Effect')
    ax.plot(XX[:, i], conf[1], 'r--', label='95% CI')
    feature_names = X.columns
    ax.set_title(f'Term {i}')
    ax.legend()
plt.tight_layout()
plt.show()